In [1]:
import pandas as pd
import numpy as np
import py_stringmatching as sm
import py_stringsimjoin as ssj
from pandas_profiling import ProfileReport
import re
import nltk
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [4]:
wor = pd.read_csv('../../datasets/books/wor.csv',sep=';')

In [5]:
half = pd.read_csv('../../datasets/books/half.csv',sep=';')

In [6]:
wor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48792 entries, 0 to 48791
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   wor_id                          48792 non-null  object
 1   wor_isbn                        48792 non-null  int64 
 2   wor_title                       48792 non-null  object
 3   wor_author                      48533 non-null  object
 4   wor_binding                     46050 non-null  object
 5   wor_pubdate                     48677 non-null  object
 6   wor_pages                       48792 non-null  int64 
 7   wor_publisher                   48744 non-null  object
 8   wor_title_no_par                48791 non-null  object
 9   wor_title_no_par_no_stop_words  48729 non-null  object
 10  wor_firstauthor                 48511 non-null  object
 11  wor_firsttwoauthors             48533 non-null  object
dtypes: int64(2), object(10)
memory usage: 4.5+ MB


In [7]:
half.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3099 entries, 0 to 3098
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   half_id                          3099 non-null   object
 1   half_isbn                        3099 non-null   int64 
 2   half_title                       3099 non-null   object
 3   half_firstauthor                 3094 non-null   object
 4   half_binding                     460 non-null    object
 5   half_pubdate                     3028 non-null   object
 6   half_pages                       3099 non-null   int64 
 7   half_publisher                   3099 non-null   object
 8   half_title_no_par                3099 non-null   object
 9   half_title_no_par_no_stop_words  3094 non-null   object
dtypes: int64(2), object(8)
memory usage: 242.2+ KB


In [8]:
#wor is not read in as string but as integer so we convert it back to string
wor['wor_isbn'] = wor['wor_isbn'].apply(lambda x: str(x))
half['half_isbn'] = half['half_isbn'].apply(lambda x: str(x))

In [9]:
wor.fillna('',inplace=True)
half.fillna('',inplace=True)

In [10]:
matches_wor_half = pd.merge(wor,half,left_on='wor_isbn',right_on='half_isbn')
print('True matches across wor and half: {}'.format(matches_wor_half.shape[0]))

True matches across wor and half: 1370


In [11]:
wor.columns

Index(['wor_id', 'wor_isbn', 'wor_title', 'wor_author', 'wor_binding', 'wor_pubdate', 'wor_pages', 'wor_publisher', 'wor_title_no_par', 'wor_title_no_par_no_stop_words', 'wor_firstauthor', 'wor_firsttwoauthors'], dtype='object')

In [12]:
half.columns

Index(['half_id', 'half_isbn', 'half_title', 'half_firstauthor', 'half_binding', 'half_pubdate', 'half_pages', 'half_publisher', 'half_title_no_par', 'half_title_no_par_no_stop_words'], dtype='object')

### Join on author based on Jaccard score of at least 0.3 with 3-gram tokenization

In [12]:
q3 = sm.QgramTokenizer(qval=3,return_set=True)

In [33]:
wor_half_pairs_a_q3_jac_03 = ssj.jaccard_join(wor, half, 'wor_id', 'half_id', 'wor_author_lower', 'half_author_lower', q3, 0.3, 
                                     l_out_attrs=['wor_title','wor_title_no_par','wor_title_no_par_no_stop_words','wor_author', 'wor_firstauthor', 'wor_firsttwoauthors','wor_binding','wor_pubdate', 'wor_pages','wor_publisher','wor_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)
#wor_half_pairs_a_q3_jac_03.to_csv('wor_half_pairs_a_q3_jac_03.csv',index=False)

### Join on first author (wor_firstauthor and half_author) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [19]:
wor_half_pairs_fa2_q3_jac_03 = ssj.jaccard_join(wor, half, 'wor_id', 'half_id', 'wor_firstauthor', 'half_author', q3, 0.3, 
                                    l_out_attrs=['wor_title','wor_title_no_par','wor_title_no_par_no_stop_words','wor_author', 'wor_firstauthor', 'wor_firsttwoauthors','wor_binding','wor_pubdate', 'wor_pages','wor_publisher','wor_isbn'], 
                                    r_out_attrs=['half_isbn','half_title','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_binding','half_pubdate','half_pages','half_publisher'],
                                    n_jobs=-3)
#wor_half_pairs_fa2_q3_jac_03.to_csv('wor_half_pairs_fa2_q3_jac_03.csv',index=False)

### Join on title (with parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [20]:
wor_half_pairs_tit_q3_jac_03 = ssj.jaccard_join(wor, half, 'wor_id', 'half_id', 'wor_title_lower', 'half_title_lower', q3, 0.3, 
                                     l_out_attrs=['wor_title','wor_title_no_par','wor_title_no_par_no_stop_words','wor_author', 'wor_firstauthor', 'wor_firsttwoauthors','wor_binding','wor_pubdate', 'wor_pages','wor_publisher','wor_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)
#wor_half_pairs_tit_q3_jac_03.to_csv('wor_half_pairs_tit_q3_jac_03.csv',index=False)

### Join on title (without parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [22]:
wor_half_pairs_titnp_q3_jac_03 = ssj.jaccard_join(wor, half, 'wor_id', 'half_id', 'wor_title_no_par', 'half_title_no_par', q3, 0.3, 
                                     l_out_attrs=['wor_title','wor_title_no_par','wor_title_no_par_no_stop_words','wor_author', 'wor_firstauthor', 'wor_firsttwoauthors','wor_binding','wor_pubdate', 'wor_pages','wor_publisher','wor_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)
wor_half_pairs_titnp_q3_jac_03.to_csv('wor_half_pairs_titnp_q3_jac_03.csv',index=False)

### Join on title (w/out parentheses) and stop words removed based on Jaccard score of at least 0.3 with 3-gram tokenization

In [25]:
wor_half_pairs_titnp_nosw_q3_jac_03 = ssj.jaccard_join(wor, half, 'wor_id', 'half_id', 'wor_title_no_par_no_stop_words', 'half_title_no_par_no_stop_words', q3, 0.3, 
                                    l_out_attrs=['wor_title','wor_title_no_par','wor_title_no_par_no_stop_words','wor_author', 'wor_firstauthor', 'wor_firsttwoauthors','wor_binding','wor_pubdate', 'wor_pages','wor_publisher','wor_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Evaluate Blocking

In [56]:
print('True matches across wor and halfdery: {}'.format(matches_wor_half.shape[0]))

True matches across wor and halfdery: 1370


In [34]:
def assignTrueLabels(df_blockedpairs, idcolumn1, idcolumn2):
    return df_blockedpairs.apply(lambda row: 1 if (row[idcolumn1]==row[idcolumn2]) else 0, axis=1)

In [35]:
def calcRedRatio(df1,df2,df_blockedpairs):
    cp = (df1.shape[0]*df2.shape[0])/2
    rr = (1-(df_blockedpairs.shape[0]/cp))*100
    return round(rr,2)

In [36]:
def calcPairCompISBN(df_matches,df_blockedpairs,true_label_col):
    pc = (df_blockedpairs[df_blockedpairs[true_label_col]==1].shape[0]/df_matches.shape[0])*100
    return round(pc,2)

In [37]:
#Assign labels to the blockedpairs df
wor_half_pairs_a_q3_jac_03['true_label'] = assignTrueLabels(wor_half_pairs_a_q3_jac_03,'l_wor_isbn','r_half_isbn')
wor_half_pairs_fa2_q3_jac_03['true_label'] = assignTrueLabels(wor_half_pairs_fa2_q3_jac_03,'l_wor_isbn','r_half_isbn')
wor_half_pairs_tit_q3_jac_03['true_label'] = assignTrueLabels(wor_half_pairs_tit_q3_jac_03,'l_wor_isbn','r_half_isbn')
wor_half_pairs_titnp_q3_jac_03['true_label'] = assignTrueLabels(wor_half_pairs_titnp_q3_jac_03,'l_wor_isbn','r_half_isbn')
wor_half_pairs_titnp_nosw_q3_jac_03['true_label'] = assignTrueLabels(wor_half_pairs_titnp_nosw_q3_jac_03,'l_wor_isbn','r_half_isbn')

In [40]:
print('Blocking based on author, Jaccaed, 3-gram, Threshold 0.3')
print('Evaluation of wor_half_pairs_a_q3_jac_03')
print('Number of rows: {:,}'.format(wor_half_pairs_a_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(wor,half,wor_half_pairs_a_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_wor_half,wor_half_pairs_a_q3_jac_03,'true_label')))
print('\n')
print('Blocking based on first author (wor) and author (half), Jaccaed, 3-gram, Threshold 0.3')
print('Evaluation of wor_half_pairs_fa2_q3_jac_03')
print('Number of rows: {:,}'.format(wor_half_pairs_fa2_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(wor,half,wor_half_pairs_fa2_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_wor_half,wor_half_pairs_fa2_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of wor_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(wor_half_pairs_tit_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(wor,half,wor_half_pairs_tit_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_wor_half,wor_half_pairs_tit_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of wor_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(wor_half_pairs_titnp_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(wor,half,wor_half_pairs_titnp_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_wor_half,wor_half_pairs_titnp_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of wor_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(wor_half_pairs_titnp_nosw_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(wor,half,wor_half_pairs_titnp_nosw_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_wor_half,wor_half_pairs_titnp_nosw_q3_jac_03,'true_label')))

Blocking based on author, Jaccaed, 3-gram, Threshold 0.3
Evaluation of wor_half_pairs_a_q3_jac_03
Number of rows: 146,851
Reduction Ratio: 99.81%
Pair Completeness: 97.3%


Blocking based on first author (wor) and author (half), Jaccaed, 3-gram, Threshold 0.3
Evaluation of wor_half_pairs_fa2_q3_jac_03
Number of rows: 49,867
Reduction Ratio: 99.93%
Pair Completeness: 84.96%


Evaluation of wor_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)
Number of rows: 19,925
Reduction Ratio: 99.97%
Pair Completeness: 82.63%


Evaluation of wor_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 5,352
Reduction Ratio: 99.99%
Pair Completeness: 63.72%


Evaluation of wor_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis and stop words, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 34,707
Reduction Ratio: 99.95%
Pair Completeness: 88.32%


In [62]:
matches_wor_half.to_csv('matching_pais/matches_wor_half.csv',index=False)

-- Blocking based on author (lower cased) with Jaccard distance using 3-gram tokens performs best.

In [39]:
wor_half_pairs_a_q3_jac_03.to_csv('filtered_correspondences/wor_half_pairs_a_q3_jac_03.csv',index=False)